Only on Google Colab, first run this code cell:

In [ ]:
! git clone https://github.com/compomics/workshop-ml-proteomics --branch EPIC-XS-workshop --depth 1
! mv workshop-ml-proteomics/* .
! rm -r workshop-ml-proteomics
! pip install -q deeplc pyteomics spectrum_utils ms2pip

Run this cell to import the required modules:

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import spectrum_utils.plot as sup

from scripts.ms2pip_utils import get_usi_spectrum, get_predicted_spectrum, get_intensity_array



### 3.1.1 Influence of precursor charge on peptide fragmentation

Let's start with a straightforward one: Depending on the precursor charge state,
the resulting fragmentation pattern will be different. This means that MS²PIP
always needs to know the precursor charge in order to predict a spectrum.

In [ ]:
top_spectrum = get_predicted_spectrum("LENNARTMARTENS", modifications="-", charge=2)
bottom_spectrum = get_predicted_spectrum("LENNARTMARTENS", modifications="-", charge=3)

plt.figure(figsize=(12,4))
plt.title("LENNARTMARTENS charge 2 vs. 3")
sup.mirror(top_spectrum, bottom_spectrum)
plt.show()

### 3.1.2 Considering peptide modification mass shifts

One of the options for MS²PIP is `modifications`. As all post-translational and artefactual peptide modifications introduce a mass shift to all fragment ions that carry the modification, MS²PIP needs to know the presense of any modifications on the peptide. This includes both "variable" and "fixed" modifications, regardless of how they were set in the search engine. 

To consider peptide modifications, MS²PIP first needs a record of modification names, the amino acids they occur on, and the corresponding mass shift that is introduced:

In [ ]:
from ms2pip.single_prediction import SinglePrediction

ms2pip = SinglePrediction(
    modification_strings=[
        "Oxidation,15.994915,opt,M",
        "Carbamidomethyl,57.021464,opt,C",
        "PhosphoS,79.966331,opt,S",
        "PhosphoT,79.966331,opt,T",
        "PhosphoY,79.966331,opt,Y",
        "iTRAQK,144.102063,opt,K",
        "iTRAQN,144.102063,opt,N-term",
    ]
)

For modifications that occur on multiple amino acids - such as phosphorylation and iTRAQ - MS²PIP requires an individual entry for each site and each name should be unique.

_Question 1.6: Note that the last modification entry `N-term` is listed instead of an amino acid. Why is that? What is the difference between a modification on the first amino acid versus a modification on the N-terminus?_

Next, MS²PIP needs to know for each peptide the location of all modifications. For this, we start counting amino acid residues from 1. The numbers `0` and `-1` are reserved for the N-terminus and the C-terminus, respectively. For example:

- `A C[Carbamidomethyl] D M K` = 2 Carbamidomethyl
- `A C[Carbamidomethyl] D M[Oxidation] K` = 2 Carbamidomethyl, 4 Oxidation
- `[Acetyl] A C[Carbamidomethyl] D M[Oxidation] K` = 0 Acetyl 2 Carbamidomethyl, 4 Oxidation

All modification names and locations are then merged with a "pipe" sign (`|`):

- `A C[Carbamidomethyl] D M K` = `2|Carbamidomethyl`
- `A C[Carbamidomethyl] D M[Oxidation] K` = `2|Carbamidomethyl|4|Oxidation`
- `[Acetyl] A C[Carbamidomethyl] D M[Oxidation] K` = `0|Acetyl|2|Carbamidomethyl|4|Oxidation`

A peptide without modifications should have the listing `-`.


Then we can run, for instance:

In [ ]:
top_spectrum = get_predicted_spectrum("LENNARTMARTENS", modifications="-", charge=2, ms2pip_instance=ms2pip)
bottom_spectrum = get_predicted_spectrum("LENNARTMARTENS", modifications="8|Oxidation", charge=2, ms2pip_instance=ms2pip)

plt.figure(figsize=(12,4))
plt.title("LENNARTMARTENS" + " unmodified vs. modified")
sup.mirror(top_spectrum, bottom_spectrum)
plt.show()

_Question: Which ions carry the modification? How much are they shifted?_


### 3.1.3 Considering factors that influence the fragmentation pattern

Before, you saw how the precursor charge changes the resulting fragmentation pattern, and how MS²PIP takes this into account. There are more factors that influence peptide fragmentation patterns, and MS²PIP provides specialized models for the most prevalent ones:

- Other fragmentation methods: trap-type CID (ion trap)
- Other instruments: Quadrupole Time-of-Flight (Sciex TripleTOF)
- Quantification labels: TMT or iTRAQ
- Special peptide properties: Alternative digest or immunopeptides

Let's start with quantification labels as an example. TMT and iTRAQ provide an ingenious method to label peptides from different samples (e.g. treatment and control), pool the samples together, identify all samples in a single run, and finally quantify peptides from each sample separately through peptide fragmentation. This can be achieved with the use of cleavable reporter ions. Each sample is labeled with a specific tag molecule with a distinct reporter ion mass. Thanks to a balance group on the whole label, the mass of the intact label is identical across samples. The result is that the peptides from each sample will group together in both MS1 and MS2 spectra, but after fragmentation, the reporter ions can be observed at distinct masses in the MS2 spectrum.


![tmt figure](https://miro.medium.com/max/1111/1*Y4yQI2eKoeR6aVDF6wJCkA.jpeg)

To predict peptides with iTRAQ labels, we need to specify two things:
- We want to use the specialized iTRAQ model, to accomodate for the altered fragmentation intensities
- We need to specify the modifications, to accomodate for the mass shifts

In [ ]:
# Setup MS²PIP instance with modification info
ms2pip = SinglePrediction(
    modification_strings=[
        "Oxidation,15.994915,opt,M",
        "iTRAQK,144.102063,opt,K",
        "iTRAQN,144.102063,opt,N-term",
    ]
)

# Predict the peptide spectrum
top_spectrum = get_predicted_spectrum("EENGVLVLNDANFDNFVADK", modifications="-", charge=2, model="HCD2021", ms2pip_instance=ms2pip)  # No TMT label
bottom_spectrum = get_predicted_spectrum("EENGVLVLNDANFDNFVADK", modifications="-1|iTRAQN", charge=2, model="iTRAQ", ms2pip_instance=ms2pip)  # With TMT label

# Plot
plt.figure(figsize=(12,4))
sup.mirror(top_spectrum, bottom_spectrum)
plt.show()

On top, you can see the predicted spectrum for the unmodified version, mirrored on the bottom the iTRAQ-labeled peptide.

_Question: Note that TMT drastically changes the fragmentation pattern. Which general trends can you see?_


### 3.1.4 Comparing predictions with observations

To use spectrum predictions to improve peptide identification, we need to be able to compare predictions to observations. Simply put, for each set of observed spectrum and predicted spectrum, we need a number that reflect how similar both spectra are. This is not straightforward, as many factors influence the performance of such a _metric_. For MS²PIP, we usually calculate the Pearson correlation of total ion current normalized, log2-transformed spectra for all b- and y-ions. Normalizing to the total ion current simply means dividing all peak intensities by the sum of all intensity in the spectrum. The sum of all peak intensities in the spectrum is then 1. Logarithmic tranformation helps to de-emphasize the effect of the few high intensity peaks on the correlation metric and increases the influence of the many low intensity peaks. In our experience, this results in a more accurate Pearson correlation. As MS²PIP only predicts b- and y-ions, we only compare observed b- and y-ions with the predictions. If peaks are absent in the observed spectrum, we assume they are zero.

Let's compare predictions of the normal model and the iTRAQ model with an observed iTRAQ spectrum. Again, using the USI, we can download an observed iTRAQ-labeled spectrum:

In [ ]:
usi = "mzspec:PXD000966:CPTAC_CompRef_00_iTRAQ_05_2Feb12_Cougar_11-10-09.mzML:scan:12298:[iTRAQ4plex]-LHFFM[Oxidation]PGFAPLTSR/3"
observed_spectrum = get_usi_spectrum(usi, modifications={0:144.102063, 4:15.994915})
observed_spectrum.annotate_peptide_fragments(0.02, "Da", "by")

plt.figure(figsize=(12,4))
plt.title(usi)
sup.spectrum(observed_spectrum)
plt.show()

As an aside, if we zoom in on the 110-120 m/z range of the spectrum, we see four peaks at equidistant masses:

In [ ]:
observed_spectrum.set_mz_range(min_mz=110, max_mz=120)

plt.figure(figsize=(12,4))
plt.title(usi)
sup.spectrum(observed_spectrum)
plt.xlim(110,120)
plt.show()

_Question: What are these four peaks?_

Ok, let's predict the spectra and compare them to the observed one. Each time, the observed iTRAQ spectrum will be on top, with the prediction WITH or WITHOUTH iTRAQ label mirrored on the bottom:

In [ ]:
# Observed spectrum
usi = "mzspec:PXD000966:CPTAC_CompRef_00_iTRAQ_05_2Feb12_Cougar_11-10-09.mzML:scan:12298:[iTRAQ4plex]-LHFFM[Oxidation]PGFAPLTSR/3"
observed_spectrum = get_usi_spectrum(usi, modifications={0:144.102063, 4:15.994915})
observed_spectrum.annotate_peptide_fragments(0.02, "Da", "by", max_ion_charge=1)

# Predict the peptide spectrum
itraq_spectrum = get_predicted_spectrum("LHFFMPGFAPLTSR", modifications="0|iTRAQN|5|Oxidation", charge=3, model="iTRAQ", ms2pip_instance=ms2pip)  # With iTRAQ label
itraq_spectrum.set_mz_range(max_mz=950)

# Plot
plt.figure(figsize=(12,4))
sup.mirror(observed_spectrum, itraq_spectrum)
plt.show()

In [ ]:
hcd_spectrum = get_predicted_spectrum("LHFFMPGFAPLTSR", modifications="5|Oxidation", charge=3, model="HCD2021", ms2pip_instance=ms2pip)  # No iTRAQ label
hcd_spectrum.set_mz_range(max_mz=950)

# Plot
plt.figure(figsize=(12,4))
sup.mirror(observed_spectrum, hcd_spectrum)
plt.show()

While there are so unannotated (black) peaks in the observed spectrum that compress the annotated peaks, the iTRAQ predictions match a lot better. Let's calculate the Pearson correlation for both comparisons:

In [ ]:
def get_processed_intensities(spectrum):
    annotated_intensities = get_intensity_array(spectrum).flatten()
    annotated_intensities = annotated_intensities / sum(annotated_intensities)
    annotated_intensities = np.log2(annotated_intensities + 0.001)
    return annotated_intensities

In [ ]:
intensities_predicted_hcd = get_processed_intensities(hcd_spectrum)
intensities_predicted_itraq = get_processed_intensities(itraq_spectrum)

observed_spectrum.annotate_peptide_fragments(0.02, "Da", "by", max_ion_charge=1)
intensities_observed = get_processed_intensities(observed_spectrum).flatten()

Prediction without iTRAQ:

In [ ]:
np.corrcoef(intensities_predicted_hcd, intensities_observed)[0][1]

Prediction with iTRAQ:

In [ ]:
np.corrcoef(intensities_predicted_itraq, intensities_observed)[0][1]

_Question: If everything went well, you can see that the Pearson correlation for the prediction with the specialized iTRAQ model is higher. This indicates that having a specialized model helps to get more accurate spectrum predictions. However, we can also turn this reasoning around: Given these two Pearson correlations, what does this tell us about the two "peptide identifications" (peptide with vs. without iTRAQ label)?_

### Extra: Using MS²PIP web server
Here we used MS²PIP locally with the Python API. However, you can also run MS²PIP for a large batch of peptides online, without any installation needs. Go to [iomics.ugent.be/ms2pip](https://iomics.ugent.be/ms2pip) and try it out! The documentation on the website should help you out.

## 3.2 More DeepLC

### 3.2.1 Effect of modifications on retention time

The following block contains a lot of code, but don't panic: It just combines
all steps from the first part of the DeepLC workshop. It goes through the
following steps: import libraries, read/transform the data, calibrate our model,
and finally make predictions.

If you do not understand this code, just ignore it and continue 😉

In [ ]:
import pandas as pd
import seaborn as sns
from deeplc import DeepLC
from matplotlib import pyplot as plt

# Suppress tensorflow logging
import os
import warnings
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
import tensorflow as tf
tf.get_logger().setLevel('ERROR')

df = pd.read_csv("https://dl.dropboxusercontent.com/s/bok4w3jw2gxohbz/deeplc_input.csv",index_col=0)
df.fillna("",inplace=True)

num_total_rows_select = 5000
num_calib = 250

sub_df_pred = df[df["scan"].isin(list(set(df["scan"].sample(num_total_rows_select))))].copy()
sub_df_calib = sub_df_pred[sub_df_pred["scan"].isin(list(set(sub_df_pred[sub_df_pred["q_value"] < 0.01]["scan"].sample(num_calib))))].copy()

sub_df_pred.rename({
    "database_peptide" : "seq",
    "rt" : "tr"
},axis=1,inplace=True)

sub_df_calib.rename({
    "database_peptide" : "seq",
    "rt" : "tr"
},axis=1,inplace=True)

dlc = DeepLC(
    cnn_model=True,
    pygam_calibration=False,
    verbose=False
)

dlc.calibrate_preds(seq_df=sub_df_calib[sub_df_calib["best_psm"]==1])

preds = dlc.make_preds(seq_df=sub_df_pred)
sub_df_pred["preds"] = preds

In the next cells we predict retention times for modified peptides:

In [ ]:
def plot_modification(sub_df_best,modification="carbamidomethyl"):
    # Init plot
    plt.figure(figsize=(7,7))
    ax = plt.gca()
    ax.set_aspect('equal')

    # Plot data
    plt.scatter(sub_df_best[sub_df_best["modifications"].str.contains(modification)]["tr"],sub_df_best[sub_df_best["modifications"].str.contains(modification)]["preds"],alpha=0.5,s=4)
    plt.plot([1500,14500],[1500,14500],c="black",linestyle="dotted")
    
    plt.title(modification)
    plt.xlabel("Observed retention time (s)")
    plt.ylabel("Predicted retention time (s)")
    
    plt.show()

In [ ]:
sub_df_best = sub_df_pred[sub_df_pred["best_psm"]==1]
sub_df_best = sub_df_best[sub_df_best["q_value"]<0.001]

plot_modification(sub_df_best,modification="carbamidomethyl")
plot_modification(sub_df_best,modification="Formyl")
plot_modification(sub_df_best,modification="Dehydrated")
plot_modification(sub_df_best,modification="Ammonium")
plot_modification(sub_df_best,modification="Sulfide")

#### 3.2.2 Playground: Design your own peptides and modifications and predict their retention time

_For the real enthusiasts!_

#### 3.2.2.1 Make predictions for your own peptide and modifications combo's

Provide the data for peptides you want to predict:

In [ ]:
#IIVINTPNNPIGK
dict_effect_aa = {
    "seq" : ["IIVINKPNNPIGK", # K on pos 6
             "IIVINTPNNPIGK", # T on pos 6
             "IIVINAPNNPIGK", # A on pos 6
             "IIVINWPNNPIGK"  # W on pos 6
            ],
    "modifications" : ["","","",""],
    "tr" : [0,1,2,3]
}

df_effect_aa = pd.DataFrame(dict_effect_aa)

In [ ]:
preds = dlc.make_preds(seq_df=df_effect_aa)

Lets have a look at their predictions:

In [ ]:
plt.scatter(df_effect_aa.index,preds)
plt.xticks(df_effect_aa.index,df_effect_aa["seq"])
plt.ylabel("Predicted retention time (s)")
plt.show()

Provide the data for peptides+modifications you want to predict:

In [ ]:
#IIVINTPNNPIGK
dict_effect_aa = {
    "seq" : ["IIVINCPNNPIGK", "IIVINCPNNPIGK", "IIVINQPNNPIGK", "IIVINQPNNPIGK", "IIVINMPNNPIGK", "IIVINMPNNPIGK"],
    "modifications" : ["","6|carbamidomethyl","","6|Deamidated","","6|Formyl"],
    "tr" : [0,1,2,3,4,5]
}

df_effect_aa = pd.DataFrame(dict_effect_aa)

In [ ]:
preds = dlc.make_preds(seq_df=df_effect_aa)

In [ ]:
plt.scatter(df_effect_aa.index,preds)
plt.xticks(df_effect_aa.index,df_effect_aa["seq"]+"+"+df_effect_aa["modifications"],rotation=90)
plt.ylabel("Predicted retention time (s)")
plt.show()

#### 3.2.2.2 Questions - playground retention time prediction

<ol>
  <li>Can you design a peptide that falls in between "IIVINKPNNPIGK" and "IIVINTPNNPIGK" in terms of retention time?</li>
  <li>What effect do certain modifications have? Is this expected?</li>
  <li>Do you expect that modifications always have the same effect?</li>
</ol>